In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import ImageGrab
from lib.SQL import SQLCalls
from keras.preprocessing.sequence import pad_sequences
try:
    xrange = xrange
except:
    xrange = range

Using TensorFlow backend.


In [2]:
def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    used = tf.sign(tf.reduce_max(tf.abs(sequence), reduction_indices=2))
    length = tf.reduce_sum(used, reduction_indices=1)
    for t in reversed(xrange(0, r.size)):
        running_add = running_add * .99 + r[t]
        discounted_r[t] = running_add   
    return discounted_r

def process_img(original_image):
    processed_img=cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
    processed_img= cv2.resize(processed_img,(28,28))
    return processed_img

In [3]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

In [8]:
def run():
    tf.reset_default_graph()
    myAgent = agent(lr=1e-2,s_size=28,a_size=4,h_size=64)
    init = tf.global_variables_initializer()
    update_frequency = 5
    init = tf.global_variables_initializer()
    d=False
    SQL=SQLCalls()
    Genomes=SQL.GatherGenomes()
    # Launch the tensorflow graph
    with tf.Session() as sess:
        sess.run(init)
        i = 0
        total_reward = []
        total_lenght = []
        UsedGenomes=np.ones(24)
        gradBuffer = sess.run(tf.trainable_variables())
        for ix,grad in enumerate(gradBuffer):
            gradBuffer[ix] = grad * 0

        while i < 1:
            s= print_screen = np.array(ImageGrab.grab(bbox=(0,60,580,530)))
            s=np.array(np.reshape(process_img(print_screen),[28,28,1]))
            running_reward = 0
            ep_history = []
            for j in range(24):
                #Probabilistically pick an action given our network outputs.
                a_dist = sess.run(myAgent.final_output,feed_dict={myAgent.genomes:Genomes,myAgent.state_in:[s]\
                                                             ,myAgent.used_genomes:UsedGenomes,myAgent.condition:0\
                                                                 ,myAgent.action_holder:[10]})
                a = np.random.choice(a_dist,p=a_dist)
                a = np.argmax(a_dist == a)
                print(a)
                if UsedGenomes[a]==-np.inf:
                    print(j)
                    print(a)
                    print(a_dist)
                    print(UsedGenomes)
                    print()
                UsedGenomes[a]=-np.inf
                #print(UsedGenomes)
                #running_reward += r
                if d == True:
                    #Update the network.
                    ep_history = np.array(ep_history)
                    ep_history[:,2] = discount_rewards(ep_history[:,2])
                    feed_dict={myAgent.reward_holder:ep_history[:,2],
                            myAgent.action_holder:ep_history[:,1],myAgent.state_in:np.vstack(ep_history[:,0])}
                    grads = sess.run(myAgent.gradients, feed_dict=feed_dict)
                    for idx,grad in enumerate(grads):
                        gradBuffer[idx] += grad

                    if i % update_frequency == 0 and i != 0:
                        feed_dict= dictionary = dict(zip(myAgent.gradient_holders, gradBuffer))
                        _ = sess.run(myAgent.update_batch, feed_dict=feed_dict)
                        for ix,grad in enumerate(gradBuffer):
                            gradBuffer[ix] = grad * 0

                    total_reward.append(running_reward)
                    total_lenght.append(j)
                    break


            i += 1
run()            

(1, 64)
(?, 64)
19
0
1
5
2
11
23
7
16
8
9
12
14
15
20
21
17
3
18
6
4
13
10
22


In [9]:
#https://gist.github.com/danijar/3f3b547ff68effb03e20c470af22c696
#https://danijar.com/variable-sequence-lengths-in-tensorflow/
def length(sequence):
    used = tf.sign(tf.reduce_max(tf.abs(sequence), reduction_indices=2))
    length = tf.reduce_sum(used, reduction_indices=1)
    length = tf.cast(length, tf.int32)
    return length
#https://gist.github.com/danijar/3f3b547ff68effb03e20c470af22c696
#https://danijar.com/variable-sequence-lengths-in-tensorflow/
def last_relevant(output, length):
    batch_size = tf.shape(output)[0]
    max_length = int(output.get_shape()[1])
    output_size = int(output.get_shape()[2])
    index = tf.range(0, batch_size) * max_length + (length - 1)
    flat = tf.reshape(output, [-1, output_size])
    relevant = tf.gather(flat, index)
    return relevant



#https://medium.com/@awjuliani/super-simple-reinforcement-learning-tutorial-part-2-ded33892c724
class agent():
    def __init__(self, lr, s_size,a_size,h_size):
        #These lines established the feed-forward part of the network. The agent takes a state and produces an action.
        #Placeholders
        self.state_in= tf.placeholder(shape=[1,s_size,s_size,1],dtype=tf.float32)
        self.genomes= tf.placeholder(shape=[24,8,3],dtype=tf.int32)       
        self.used_genomes= tf.placeholder(shape=[24],dtype=tf.int32)
        self.condition = tf.placeholder(tf.int32, shape=[], name="condition")
        self.training=tf.placeholder(shape=[1],dtype=tf.bool)
        self.reward_holder = tf.placeholder(shape=[None],dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[None],dtype=tf.int32)
        #Convolution Network 
        conv1 = tf.layers.conv2d(inputs=self.state_in,filters=32,kernel_size=[5, 5],padding="same",activation=tf.nn.relu)
        pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
        conv2 = tf.layers.conv2d(inputs=pool1,filters=64,kernel_size=[5, 5],padding="same",activation=tf.nn.relu)
        pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
        flatten= tf.reshape(pool2,[-1,7*7*64])
        hidden_conv = slim.fully_connected(flatten,64,biases_initializer=None,activation_fn=tf.nn.relu)
        
        #Recurrent Neural Network
        sequence_output, state = tf.nn.dynamic_rnn(tf.contrib.rnn.LSTMCell(64),\
                                          tf.cast(self.genomes,tf.float32),dtype=tf.float32,sequence_length=length(self.genomes))
        last = last_relevant(sequence_output, length(self.genomes)) 
        hidden_rnn= slim.fully_connected(last,64,biases_initializer=None,activation_fn=tf.nn.relu)
        
        #Training/Testing Changes
        #Clone Image by Genome Size
        print(hidden_conv.get_shape())
        hidden_conv = tf.cond(self.condition <  1, lambda: tf.tile(hidden_conv,[24,1]), lambda: hidden_conv)
        print(hidden_conv.get_shape())
        #Change Recurrent Net to Match Inputs
        hidden_rnn=tf.cond(self.condition < 1, lambda: hidden_rnn,lambda: tf.gather(hidden_rnn,self.action_holder))
        #tf.gather(hidden_rnn,self.action_holder)
        
        combined=tf.concat([hidden_rnn,hidden_conv],1)
        hidden_combined= slim.fully_connected(combined,1024,biases_initializer=None,activation_fn=tf.nn.relu)
        self.regression= slim.fully_connected(hidden_combined,1,biases_initializer=None,activation_fn=tf.nn.sigmoid)
        self.output=tf.reshape(self.regression,[-1])*tf.cast(self.used_genomes,tf.float32)
        self.final_output=tf.nn.softmax(self.output)

        #self.chosen_action = tf.argmax(self.output,1)

        
        
        
        #The next six lines establish the training proceedure. We feed the reward and chosen action into the network
        #to compute the loss, and use it to update the network.


        self.loss = -tf.reduce_mean(tf.log(self.output)*self.reward_holder)
        
        tvars = tf.trainable_variables()
        self.gradient_holders = []
        for idx,var in enumerate(tvars):
            placeholder = tf.placeholder(tf.float32,name=str(idx)+'_holder')
            self.gradient_holders.append(placeholder)
        
        self.gradients = tf.gradients(self.loss,tvars)
        
        optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        self.update_batch = optimizer.apply_gradients(zip(self.gradient_holders,tvars))
run()

(1, 64)
(?, 64)
12
4
0
3
19
1
18
5
11
10
8
2
21
7
9
23
16
6
14
15
20
22
17
13


In [ ]:
a_dist=[.3,.2,.5]
print(a_dist)
a=np.random.choice(a_dist,p=a_dist)
print(a)
b=np.argmax([a_dist] == a)
print(b)
c=np.argmax(a_dist == a)
print(c)